In [1]:
import pandas as pd

In [2]:
from sklearn.metrics import cohen_kappa_score

In [3]:
coding_ob = pd.read_csv("../input/answers_ob.csv", index_col=0).reset_index(drop=True)
coding_bu = pd.read_csv("../input/answers_bu.csv", index_col=0).reset_index(drop=True)

In [4]:
coding_ob = coding_ob.drop_duplicates(subset=["question_id", "answer_correct", "content"])
coding_bu = coding_bu.drop_duplicates(subset=["question_id", "answer_correct", "content"])

In [5]:
#coding_bu[["is_correct", "has_correct_reasoning"]] = coding_bu[["is_correct", "has_correct_reasoning"]].fillna(False)

In [6]:
coding_ob.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1373 entries, 0 to 1383
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   question_id            1373 non-null   object
 1   question_language      1373 non-null   object
 2   question_tom_type      1373 non-null   object
 3   question_tom_order     1373 non-null   int64 
 4   question_type          1373 non-null   object
 5   answer_correct         1373 non-null   object
 6   content                1373 non-null   object
 7   is_correct             1373 non-null   bool  
 8   has_correct_reasoning  1106 non-null   object
 9   is_beyond_scope        101 non-null    object
 10  has_grammatical_error  46 non-null     object
 11  needs_debating         95 non-null     object
dtypes: bool(1), int64(1), object(10)
memory usage: 130.1+ KB


In [7]:
coding_bu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1342 entries, 0 to 1351
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   question_id            1342 non-null   object
 1   question_language      1342 non-null   object
 2   question_tom_type      1342 non-null   object
 3   question_tom_order     1342 non-null   int64 
 4   question_type          1342 non-null   object
 5   answer_correct         1342 non-null   object
 6   content                1342 non-null   object
 7   is_correct             1342 non-null   bool  
 8   has_correct_reasoning  1081 non-null   object
 9   is_beyond_scope        83 non-null     object
 10  has_grammatical_error  2 non-null      object
 11  needs_debating         44 non-null     object
dtypes: bool(1), int64(1), object(10)
memory usage: 127.1+ KB


In [8]:
key_columns = [
    "question_id",
    "answer_correct",
    "content"
]

In [9]:
columns_to_compare = [
    "is_correct",
    "has_correct_reasoning",
    "is_beyond_scope"
]

In [10]:
columns_to_exclude = [
    column
    for column
    in coding_ob
    if column not in columns_to_compare
    and column not in key_columns
]

In [11]:
columns_to_exclude

['question_language',
 'question_tom_type',
 'question_tom_order',
 'question_type',
 'has_grammatical_error',
 'needs_debating']

In [12]:
coding_merged = coding_ob.drop(
    columns=columns_to_exclude).merge(
    coding_bu.drop(columns=columns_to_exclude),
    on=key_columns,
    suffixes=("_ob", "_bu")
    )

In [13]:
coding_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1342 entries, 0 to 1341
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   question_id               1342 non-null   object
 1   answer_correct            1342 non-null   object
 2   content                   1342 non-null   object
 3   is_correct_ob             1342 non-null   bool  
 4   has_correct_reasoning_ob  1086 non-null   object
 5   is_beyond_scope_ob        101 non-null    object
 6   is_correct_bu             1342 non-null   bool  
 7   has_correct_reasoning_bu  1081 non-null   object
 8   is_beyond_scope_bu        83 non-null     object
dtypes: bool(2), object(7)
memory usage: 76.1+ KB


In [14]:
coding_merged.head(3)

,question_id,answer_correct,content,is_correct_ob,has_correct_reasoning_ob,is_beyond_scope_ob,is_correct_bu,has_correct_reasoning_bu,is_beyond_scope_bu
0,1_EN,chocolate,Sam believes that the bag is full of chocolate.,True,NaN,NaN,True,NaN,NaN
1,1_EN,chocolate,chocolate.,True,NaN,NaN,True,NaN,NaN
2,1_EN,chocolate,chocolate,True,NaN,NaN,True,NaN,NaN


In [15]:
coding_ob.head(3)

,question_id,question_language,question_tom_type,question_tom_order,question_type,answer_correct,content,is_correct,has_correct_reasoning,is_beyond_scope,has_grammatical_error,needs_debating
0,1_EN,English,Belief,1,Closed-ended,chocolate,Sam believes that the bag is full of chocolate.,True,NaN,NaN,NaN,NaN
1,1_EN,English,Belief,1,Closed-ended,chocolate,chocolate.,True,NaN,NaN,NaN,NaN
2,1_EN,English,Belief,1,Closed-ended,chocolate,chocolate,True,NaN,NaN,NaN,NaN


In [16]:
coding_merged = pd.concat(
    [coding_merged,
     coding_ob[columns_to_exclude
    ].reset_index(drop=True)], axis=1
    )

In [17]:
coding_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1373 entries, 0 to 1372
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   question_id               1342 non-null   object
 1   answer_correct            1342 non-null   object
 2   content                   1342 non-null   object
 3   is_correct_ob             1342 non-null   object
 4   has_correct_reasoning_ob  1086 non-null   object
 5   is_beyond_scope_ob        101 non-null    object
 6   is_correct_bu             1342 non-null   object
 7   has_correct_reasoning_bu  1081 non-null   object
 8   is_beyond_scope_bu        83 non-null     object
 9   question_language         1373 non-null   object
 10  question_tom_type         1373 non-null   object
 11  question_tom_order        1373 non-null   int64 
 12  question_type             1373 non-null   object
 13  has_grammatical_error     46 non-null     object
 14  needs_debating          

In [33]:
def compare_columns(coding_merged: pd.DataFrame, columns_to_compare):
    output = {item: {} for item in columns_to_compare}
    for column in columns_to_compare:
        column_ob = f"{column}_ob"
        column_bu = f"{column}_bu"

        agreement_series = coding_merged[column_ob] == coding_merged[column_bu]
        nan_agreement_series = coding_merged[column_ob].isna() & coding_merged[column_bu].isna()

        total_agreement_series = agreement_series | nan_agreement_series

        #matches = (coding_merged[column_ob] == coding_merged[column_bu]).sum()
        percentage_agreement = total_agreement_series.sum() / len(coding_merged)

        kappa_score = cohen_kappa_score(
            coding_merged[column_ob].astype("Int64").fillna(-1),
            coding_merged[column_bu].astype("Int64").fillna(-1)
        )

        output[column]["agreement_percentage"] = percentage_agreement
        output[column]["cohen_kappa"] = kappa_score
    
    return output

In [24]:
comparison = compare_columns(
    coding_merged.loc[
        (coding_merged["question_type"] == "Closed-ended")
        & 
        (True)], columns_to_compare)

In [37]:
for key, value in comparison.items():
    print(f"The '{key}' column has\n\t{round(value["agreement_percentage"] * 100, 2)}% agreement with a\n\tCohen's kappa coefficient of {round(value["cohen_kappa"], 3)}.")

The 'is_correct' column has
	98.74% agreement with a
	Cohen's kappa coefficient of 0.973.
The 'has_correct_reasoning' column has
	94.97% agreement with a
	Cohen's kappa coefficient of 0.906.
The 'is_beyond_scope' column has
	96.86% agreement with a
	Cohen's kappa coefficient of 0.837.
